In [1]:
import torch
import torch.nn as nn

In [2]:
filepath = 'vanznames.txt'

In [3]:
with open(filepath, 'r') as f:
    names_raw = list(map(lambda x: x.replace('\n', ''), f.readlines()))
    
all_letters = ['<SOS>'] + list(set("".join(names_raw))) + ['<EOS>']
n_letters = len(all_letters)
sos_idx = 0
eos_idx = n_letters - 1
                
def encode_name(name):
    return [sos_idx] + [all_letters.index(s) for s in name] + [eos_idx]

names_enc = [encode_name(name) for name in names_raw]

In [4]:
print(n_letters)
print(names_enc[0])

160
[0, 129, 124, 62, 140, 143, 65, 128, 118, 9, 126, 118, 155, 118, 47, 155, 129, 118, 65, 123, 75, 159]


In [5]:
# define network
class VanzNet(nn.Module):
    def __init__(self, n_letters, n_dim, hidden_size, dor=0.15, bid=False):
        super(VanzNet, self).__init__()
        self.hidden_size = hidden_size
        self.emb = nn.Embedding(n_letters, n_dim)
        self.lstm = nn.LSTM(n_dim, hidden_size, num_layers=2
                            , dropout=dor, bidirectional=bid)
        if bid:
            input_mult = 2
        else:
            input_mult = 1
        
        self.lstm2o = nn.Linear(hidden_size*input_mult, n_letters)
    
    def forward(self, src, h=None, c=None):
        src_emb = self.emb(src)
        if (h is not None) and (c is not None):
            output_lstm, (h_out, c_out) = self.lstm(src_emb, (h, c))
        else:
            output_lstm, (h_out, c_out) = self.lstm(src_emb)
        output_fc = self.lstm2o(output_lstm)
        return output_fc, h_out, c_out

In [6]:
# one-hot matrix for characters
def name2input(name_str):
    return torch.LongTensor(encode_name(name_str)[:-1]).view(-1, 1)

def name2target(name_enc):
    """
    dim: [len_name]
    e.g. target(KASPAROV) -> ASPAROV<EOS> -> [Idx(A), Idx(S), ..., Idx(EOS)]
    """
    return torch.LongTensor(name_enc[1:] + [n_letters - 1])

In [7]:
hidden_size = 64
n_dim = 32
rnn = VanzNet(n_letters, n_dim, hidden_size)

In [26]:
def reconstruct_char(output):
    idx = output.argmax(2)[-1, 0]
    # print(idx)
    if idx == n_letters - 1:
        # return all_letters[topi[0][0][1].item()]
        return 'EOS'
    else:
        return all_letters[idx]

def sample_name(model, start_char):
    model.eval()
    name_sample = start_char
    if not (start_char[0] in all_letters):
        return "Invalid start character!"
    else:
        input_tensor = name2input(start_char)
        out_char = ""
        h = None
        c = None
        # print(type(input_tensor))
        while True and len(name_sample) < 40:
            output, _, _ = model(input_tensor)
            out_char = reconstruct_char(output)
            if out_char == 'EOS':
                break
            name_sample += out_char
            input_tensor = name2input(name_sample)
    
    return name_sample

In [9]:
sample_name(rnn, 'ก')

'กយyyyyyyyyyyyyyyyyyy'

In [10]:
lr = 1e-3
epochs = 200
print_every = 5
max_training_size = -1
lr_decay_step = 30
lr_decay_rate = 0.95

In [11]:
if max_training_size > 0:
    names_train = names_enc[:max_training_size]
else:
    names_train = names_enc

criterion = nn.CrossEntropyLoss()
optim = torch.optim.Adam(rnn.parameters(), lr=lr)

scheduler = torch.optim.lr_scheduler.StepLR(optim
                                            , lr_decay_step
                                            , lr_decay_rate)

In [12]:
import time
import random

In [13]:
losses = []
for epoch in range(epochs):
    loss_epoch = 0
    epoch_start = time.time()
    for name in names_train:
        input_tensor = torch.LongTensor(name[:-1]).view(-1, 1)
        # target_tensor = name2target(name)
        target_tensor = torch.LongTensor(name[1:])
        optim.zero_grad()
        
        output, _, _ = rnn(input_tensor)
        # print(output.shape, target_tensor.shape)
        loss = criterion(output.view(-1, n_letters), target_tensor)
        loss.backward()
        optim.step()
        loss_epoch += loss.item()
    
    epoch_end = time.time()
    scheduler.step()
    losses.append(loss_epoch / len(names_train))
    random.shuffle(names_train)
    if (epoch + 1) % print_every == 0:
        print("%d/%d: Loss %f, %.2f sec" % (epoch+1, epochs, loss_epoch, epoch_end-epoch_start))
        print(sample_name(rnn, 'ก'))
        print(sample_name(rnn, 'ค'))
        # print(output.topk(1)[1])
        # print(target_tensor)
        # print("----------------")

5/200: Loss 5408.666836, 27.82 sec
กา' เอ็ก
คน' เดียว
10/200: Loss 4705.320850, 28.30 sec
กระพันธ์ เทืองเดิน
คน เอ็ม
15/200: Loss 4222.370506, 28.03 sec
กู เอ เอ เอ เอ เอ เอ
คน เมือง
20/200: Loss 3840.574968, 28.36 sec
กิตติ พิมมันทร์
คน' เด็กกรุ่ง
25/200: Loss 3536.840122, 28.41 sec
กิตติพัฒน์ นวรรณ์
คนเดียว ร้าย'ยย
30/200: Loss 3298.723572, 28.69 sec
กรูส์ เมียก
คนเก้า หมู
35/200: Loss 3066.283795, 28.90 sec
กู เอ  คนเดิม
คน'นิน สาย'ย่อย
40/200: Loss 2905.739683, 28.97 sec
กรกฤต เรืองรักษ์
คน บ้านนอก
45/200: Loss 2765.450769, 28.67 sec
กู เจ๋ง
คน บ้านนอก
50/200: Loss 2662.130733, 28.15 sec
กรุณา มูน
คนเดิ น๋นน
55/200: Loss 2566.526863, 28.71 sec
กรกฤต เรืองรักษ์
คน' ก.
60/200: Loss 2484.940969, 28.39 sec
กรกฤต บัวเพ็ชเเยบ
คน บ้า
65/200: Loss 2390.897033, 28.18 sec
กรวินท์ สมบุญ
คน บ้านกลาง
70/200: Loss 2316.651729, 28.25 sec
กรุอบ น้ำพาไต
คน บ้านนอก
75/200: Loss 2284.672038, 28.81 sec
กู เจ๋ง
คน' กู
80/200: Loss 2232.646546, 30.82 sec
กิตติพร วิญญา
คน บ้า บ้าตาก
85/200: Loss 2195.488

In [14]:
start_letters = ['ก', 'ค', 'ม', 'จ', 'ว', 'ร', 'บ']
# start_letters = all_letters

with torch.no_grad():
    for sp in start_letters:
        print(sample_name(rnn, sp))

กู เจ๋ง
คนชั่วคราว พระเมืองต
มอส คุง
จิระพันธ์ รุงทรงทอง
วิมร้า ไมทฑวงคม
รักเดียว 'ยิ้ง หนหนห
บอล อังกฤษ


In [35]:
start_letters = ['บอย', 'ฟ้า', 'ลุง', 'จอน', 'เต้ย', 'โอ๊ต', 'โอ ', 'เอิร์ธ']
# start_letters = all_letters

with torch.no_grad():
    for sp in start_letters:
        print(sample_name(rnn, sp))

บอย กลับ' ต้า
ฟ้า เถื่อนน
ลุงพ์ เทพนัน
จอน จัด
เต้ย จัดหั้ย
โอ๊ต หนองใหล่สับ
โอ คนเดิม นะครับ
เอิร์ธ' เก่ง
